Before you turn in your homework, make sure everything runs as expected.

Make sure you execute every single code cell, in order, filling with your solutions in any place that says `# YOUR CODE HERE`, and always DELETE the line that says:

```python
raise NotImplementedError()
```

The purpose of this line is to tell you if you forgot to answer a question (it will throw an error if the line is there)

**IMPORTANT:**

* **DO NOT DELETE ANY CELL** and do not change the title of the Notebook.

* Use the same variable names as the ones written in the questions; otherwise, the tests will fail.

* Before you turn in your homework, make sure everything runs as expected: restart the kernel (in the menubar, select Kernel $\rightarrow$ Restart) and then run all cells (in the menubar, select Cell $\rightarrow$ Run All).

Fill your name below:

In [ ]:
name = ""

###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2014 L.A. Barba, C.D. Cooper, G.F. Forsyth.

# Assignment 4: Reaction-diffusion model

This Jupyter Notebook presents the context and set-up for the coding assignment of Module 4: _Spreading out: Diffusion problems_, of the course [**"Practical Numerical Methods with Python"**](https://github.com/numerical-mooc/numerical-mooc) (a.k.a., numericalmooc).

So far in this module, we've studied diffusion in 1D and 2D. Now it's time to add in some more interesting physics.  You'll study a model represented by *reaction-diffusion* equations.  What are they?  The name says it all—it's a system that has the physics of diffusion but also has some kind of reaction that adds different behaviors to the solution.

We're going to look at the _Gray-Scott model_, which simulates the interaction of two generic chemical species reacting and ... you guessed it ... diffusing! Some amazing patterns can emerge with simple reaction models, eerily reminiscent of patterns formed in nature. It's fascinating! Check out this simulation by Karl Sims posted on You Tube ... it looks like a growing coral reef, doesn't it?

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('8dTmUr5qKvI')

---

## Gray-Scott model

The Gray-Scott model represents the reaction and diffusion of two generic chemical species, $U$ and $V$, whose concentration at a point in space is represented by variables $u$ and $v$. The model follows some simple rules.  

* Each chemical _diffuses_ through space at its own rate.
* Species $U$ is added at a constant feed rate into the system.
* Two units of species V can 'turn' a unit of species U into V: $\; 2 V + U \rightarrow 3 V$
* There's a constant kill rate removing species $V$.

This model results in the following system of partial differential equations for the concentrations $u(x,y,t)$ and $v(x,y,t)$ of both chemical species:

$$
\begin{align}
    \frac{\partial u}{\partial t} &= D_u \nabla^2 u - u v^2 + F (1 - u)\\
    \frac{\partial v}{\partial t} &= D_v \nabla^2 v + u v^2 - (F + k) v
\end{align}
$$

You should see some familiar terms, and some unfamiliar ones.  On the left-hand side of each equation, we have the time rate of change of the concentrations. The first term on the right of each equation correspond to the spatial diffusion of each concentration, with $D_u$ and $D_v$ the respective rates of diffusion.

In case you forgot, the operator $\nabla ^2$ is the Laplacian:

$$
\nabla ^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial ^2 u}{\partial y^2}
$$

The second term on the right-hand side of each equation corresponds to the reaction. You see that this term decreases $u$ while it increases $v$ in the same amount: $uv^2$.  The reaction requires one unit of $U$ and two units of $V$,  resulting in a reaction rate proportional to the concentration $u$ and to the square of the concentration $v$. This result derives from the _law of mass action_, which we can explain in terms of probability: the odds of finding one molecule of species $U$ at a point in space is proportional to the concentration $u$, while the odds of finding two molecules of $V$ is proportional to the concentration squared, $v^2$. We assume here a reaction rate constant equal to $1$, which just means that the model is non-dimensionalized in some way.

The final terms in the two equations are the "feed" and "kill" rates, respectively: $F(1-u)$ replenishes the species $U$ (which would otherwise run out, as it is being turned into $V$ by the reaction); $-(F+k)v$ is diminishing the species $V$ (otherwise the concentration $v$ would simply increase without bound). 

The values of $F$ and $k$ are chosen parameters and part of the fun of this assignment is to change these values, together with the diffusion constants, and see what happens.

---

## Problem setup

The system is represented by two arrays, `U` and `V`, holding the discrete values of the concentrations $u$ and $v$, respectively.  We start by setting `U = 1` everywhere and `V = 0` everywhere, then introduce areas of difference, as initial conditions.  We then add a little noise to the whole system to help the $u$ and $v$ reactions along.  

Below is the code segment we used to generate the initial conditions for `U` and `V`.  

**NOTE**: *DO NOT USE THIS CODE IN YOUR ASSIGNMENT*.
We are showing it here to help you understand how the system is constructed.  However, you _must use the data we've supplied below_ as your starting condition or your answers will not match those that the grading system expects.

```python
def create_initial_solution(n, num_blocks, r, v1, v2, noise):
    randx = numpy.random.randint(1, n - 1, num_blocks)
    randy = numpy.random.randint(1, n - 1, num_blocks)
    u0 = numpy.ones((n, n))
    v0 = numpy.zeros((n, n))
    for i, j in zip(randx, randy):
        i1, i2 = i - r, i + r
        j1, j2 = j - r, j + r
        u0[j1:j2, i1:i2] = v1
        v0[j1:j2, i1:i2] = v2
    u0 += noise * numpy.random.random((n, n))
    v0 += noise * numpy.random.random((n, n))
    return u0, v0

u0, v0 = create_initial_solution(nx, 40, 10, 0.50, 0.25, 0.05)
filepath = 'uvinitial.npz'
with open(filepath, 'wb') as outfile:
    numpy.savez(outfile, U=u0, V=v0)
```

---

## Initial condition data files

We are going to load the initial solution of the species concentrations from the file `uvinitial.npz`.
This is a NumPy save-file that contains two NumPy arrays, holding the initial values for `U` and `V`, respectively.  You can load and plot the initial data using the following code snippets.

In [ ]:
import numpy
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
# Load the initial data from file.
filepath = 'uvinitial.npz'
with open(filepath, 'rb') as infile:
    data = numpy.load(infile)
    u0, v0 = data['U'], data['V']

In [ ]:
# Plot the initial data.
fig, ax = pyplot.subplots(ncols=2, figsize=(9.0, 4.0))
ax[0].set_title('$u$')
ax[0].imshow(u0, cmap=cm.Greys_r)
ax[0].axis('off')
ax[1].set_title('$v$')
ax[1].imshow(v0, cmap=cm.Greys_r)
ax[1].axis('off');

---

## Your assignment

Compute the solution of the reaction-diffusion equations using the **midpoint method** (also called modified Euler's method) using a **central-difference discretization** for the spatial second-order derivatives.

**Parameters:**

* Rates of diffusion: $D_u = 0.00016$, $D_v = 0.00008$.
* Feed and kill parameters: $F = 0.035$, $k = 0.060$.
* Domain: $5 \rm m \times 5 \rm m$.
* Uniform grid with $192 \times 192$ points ($\Delta x = \Delta y = \delta$).
* Time-step size: $\Delta t = \frac{9}{40} \frac{\delta^2}{\max \left( D_u, D_v \right)}$.
* Number of time steps: $nt = 1000$.
* Use zero-gradient Neumann boundary conditions on all sides of the domain.

Below is an animated gif showing the solution over $1000$ time steps (with $100$ time steps between two consecutive frames).

In [ ]:
from IPython.display import Image
Image(filename="figures/zebrafish.gif", width=400, height=400)

---
## Implement your solution (50 points)

Implement your solution in this section.
You can use as many code cells as you want.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

---

## Assessment

Answer questions in this section.

Do not try to delete or modify the cells that are already present.
For each question, make sure to provide your answer in the cell **just above** the empty cell.
(This empty cell contains hidden tests to assert the correctness of your answer and cannot be deleted.)

In [ ]:
import mooc

* Q1 (10 points): Plot the concentration of the species $U$ at the final time (i.e., after $1000$ time steps)?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

* Q2 (10 points): After $1000$ time steps, what is the NumPy array returned by `u[100, ::40]` (replacing `u` with whatever the name of your $u$ array is)?

Store your result in the variable `u_sample`; you can check your answer by calling the function `mooc.check('hw4_answer2', u_sample)`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

* Q3 (10 points): After $1000$ time steps, what is the minimum of the concentration for the $U$ species?

Store your result in the variable `u_min`; you can check your answer by calling the function `mooc.check('hw4_answer3', u_min)`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

* Q4 (10 points): After $1000$ time steps, what are the row and column indices of the minimum concentration for the $U$ species?

Store the row index in the variable `row` and the column index in the variable `col`; you can check your answer by calling the function `mooc.check('hw4_answer4', row, col)`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

* Q5 (10 points): After $1000$ time steps, what is the maximum value in the $91^{\rm {th}}$ row of the 2D array with the concentration for the $V$ species?

Store your answer in the variable `v_max`; you can check your answer by calling the function `mooc.check('hw4_answer5', v_max)`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

---

## Exploring extra patterns

Once you have completed the assignment, you might want to explore a few more of the interesting patterns that can be obtained with the Gray-Scott model.  The conditions below will result in a variety of patterns and should work without any other changes to your existing code.

In [ ]:
#Du, Dv, F, k = 0.00014, 0.00006, 0.035, 0.065 # Bacteria 2
#Du, Dv, F, k = 0.00016, 0.00008, 0.060, 0.062 # Coral
#Du, Dv, F, k = 0.00019, 0.00005, 0.060, 0.062 # Fingerprint
#Du, Dv, F, k = 0.00010, 0.00010, 0.018, 0.050 # Spirals
#Du, Dv, F, k = 0.00012, 0.00008, 0.020, 0.050 # Spirals Dense
#Du, Dv, F, k = 0.00010, 0.00016, 0.020, 0.050 # Spirals Fast
#Du, Dv, F, k = 0.00016, 0.00008, 0.020, 0.055 # Unstable
#Du, Dv, F, k = 0.00016, 0.00008, 0.050, 0.065 # Worms 1
#Du, Dv, F, k = 0.00016, 0.00008, 0.054, 0.063 # Worms 2
#Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.060 # Zebrafish

---

## References

*  Reaction-diffusion tutorial, by Karl Sims
http://www.karlsims.com/rd.html

*  Pearson, J. E. (1993). [Complex patterns in a simple system](http://www.sciencemag.org/content/261/5118/189), _Science_, Vol. 261(5118), 189-192 // [PDF](http://www3.nd.edu/~powers/pearson.pdf) from nd.edu.

*  Pattern Parameters from [http://www.aliensaint.com/uo/java/rd/](http://www.aliensaint.com/uo/java/rd/)

---
###### The cell below loads the style of the notebook

In [ ]:
from IPython.core.display import HTML
css_file = './styles/numericalmoocstyle.css'
HTML(open(css_file, 'r').read())